In [8]:
# pip install paramiko

In [2]:
import boto3
import pandas as pd
import json
import numpy as np
import time
import paramiko
import io

In [3]:
client = boto3.client('ec2', region_name='us-east-1')
# Create SQS client
sqs = boto3.resource('sqs')

In [20]:
def get_default_security_group(client, key_name):
    #extract key_name attribute from the security groups returned
    response = [group[key_name] for group in client.describe_security_groups()['SecurityGroups'] if group['GroupName'] == 'default']
    
    return response
    
def get_key_pairs(client, removeExisting=False):
    if removeExisting:
        client.delete_key_pair(KeyName='airscholar-key')
        
    keypairs = client.describe_key_pairs()['KeyPairs']
    keypair = list(filter(lambda x: x['KeyName'] == 'airscholar-key', keypairs))
    
    if not keypair:
        keypair = client.create_key_pair(KeyName='airscholar-key')
        f = io.StringIO(keypair['KeyMaterial'])
        data = f.read()
        file = open('labsuser.pem', 'w')
        file.write(data)
        file.close()
    else: 
        keypair = keypair[0]
    
    return keypair

def launch_new_instance(client, keypair):
    response = client.run_instances(
        ImageId='ami-05723c3b9cf4bf4ff',
        InstanceType='t2.micro',
        KeyName=keypair,
        MaxCount=1,
        MinCount=1,
        Monitoring={
            'Enabled': True
        },
        SecurityGroupIds= get_default_security_group(client, key_name='GroupId')
    )
    ec2_inst_id=response["Instances"][0]['InstanceId']
    waiter = client.get_waiter('instance_running')
    waiter.wait(InstanceIds=[ec2_inst_id])
    return ec2_inst_id

def prepare_instances(keypair):
    ec2 = boto3.resource('ec2')
    ec2_inst_ids = []
    for instance in ec2.instances.all():
        if instance.state['Name'] == 'running':
            ec2_inst_ids.append(instance.id)

    if not ec2_inst_ids:
        ec2_inst_ids.append(launch_new_instance(keypair))

    return ec2, ec2_inst_ids

def configure_ssh():
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    return ssh

def ssh_connect_with_retry(ssh, ip_address, retries):
    if retries > 3:
        return False
    f = open('labsuser.pem', 'r')
    privkey = paramiko.RSAKey.from_private_key(f)
    # print(privkey)
    interval = 5
    try:
        retries += 1
        print('SSH into the instance: {}'.format(ip_address))
        ssh.connect(hostname=ip_address,
                    username='ec2-user', pkey=privkey)
        return True
    except Exception as e:
        print(e)
        time.sleep(interval)
        print('Retrying SSH connection to {}'.format(ip_address))
        ssh_connect_with_retry(ssh, ip_address, retries)
        
def get_public_address(ec2, instance_id):
    # ec2 = boto3.resource('ec2', region_name='us-east-1')
    instance = ec2.Instance(id=instance_id)
    instance.wait_until_running()
    current_instance = list(ec2.instances.filter(InstanceIds=[instance_id]))
    ip_address = current_instance[0].public_ip_address
    return ip_address

def get_queue(sqs, queue_name):  
    # Get the queue. This returns an SQS.Queue instance
    queue = sqs.get_queue_by_name(QueueName=queue_name)

    if not queue:
        # Create a SQS queue
        response = sqs.create_queue(
            QueueName=queue_name,
            Attributes={
                'DelaySeconds': '60',
                'MessageRetentionPeriod': '86400'
            }
        )
        queue_url = response['QueueUrl']
    else:
        queue_url = queue.url
    
    return queue_url

def send_message_to_queue(sqs, queue_url, message, instance_id, matrix_index):
    queue = sqs.get_queue_by_name(QueueName=QUEUE_NAME)

    # Send message to SQS queue
    response = queue.send_messages(
        Entries=[
            {
                'Id': '1',
                'MessageBody': 'world'
            },
            {
                'Id': '2',
                'MessageBody': 'boto3',
                'MessageAttributes': {
                    'Author': {
                        'StringValue': 'Daniel',
                        'DataType': 'String'
                    }
                }
            }
        ]
    )
    return response

def install_required_packages(ssh):
    stdin, stdout, stderr = ssh.exec_command("sudo yum install pip -y && pip install numpy")
    return stdout, stderr

def get_messages_from_queue(queue_url):
    sqs_client = boto3.client('sqs')

    messages = []

    while True:
        resp = sqs_client.receive_message(
            QueueUrl=queue_url
        )

        try:
            messages.extend(resp['Messages'])
        except KeyError:
            break

        entries = [
            {'Index': msg['MatrixId'], 'Matrix': msg['Matrix']}
            for msg in resp['Messages']
        ]

        resp = sqs_client.delete_message_batch(
            QueueUrl=queue_url, Entries=entries
        )

        if len(resp['Successful']) != len(entries):
            raise RuntimeError(
                f"Failed to delete messages: entries={entries!r} resp={resp!r}"
            )

    return messages

def split(array, nrows, ncols):
    """Split a matrix into sub-matrices."""

    r, h = array.shape
    return (array.reshape(h//nrows, nrows, -1, ncols)
                 .swapaxes(1, 2)
                 .reshape(-1, nrows, ncols))

def generate_array(nrows, ncols, split_size):
    arr = np.random.randint(10, size=(nrows, ncols))
    arr = split(arr, split_size, split_size)
    arr1 = np.random.randint(10, size=(nrows, ncols))
    arr1 = split(arr1, split_size, split_size)
    
    return arr, arr1

In [21]:
QUEUE_NAME = 'airscholar-queue'

In [22]:
queue_url = get_queue(sqs, QUEUE_NAME)

In [23]:
ssh = configure_ssh() 
keypair = get_key_pairs(client, False)
ec2, instances = prepare_instances(keypair['KeyName'])
ip_address = get_public_address(ec2, instances[0])
ssh_connect_with_retry(ssh, ip_address, 0)

SSH into the instance: 18.212.10.19


True

In [ ]:
# pip install numpy

In [24]:
stdout, stderr = prepare_required_packages(ssh)
print('stdout:', stdout.read())
# print('stderr:', stderr.read())

NameError: name 'prepare_required_packages' is not defined

In [150]:
# sqs = boto3.resource('sqs')
# sqs

In [ ]:
# response = send_message_to_queue(sqs, QUEUE_NAME, "Hi there! This is my queue message4!", 1, 0)
response = send_message_to_queue(sqs, QUEUE_NAME, 'X', 1, 0)

In [193]:
# print(json.dumps(response['Successful'], indent=4))

In [221]:
arr, arr1 = generate_array(100,100, 5)

In [223]:
arr

array([[[8, 1, 2, 9, 3],
        [3, 1, 9, 7, 3],
        [5, 0, 1, 3, 8],
        [9, 1, 6, 0, 7],
        [3, 3, 8, 4, 4]],

       [[9, 5, 4, 7, 7],
        [9, 4, 5, 5, 1],
        [2, 3, 0, 7, 5],
        [6, 5, 7, 6, 9],
        [3, 2, 1, 2, 1]],

       [[7, 8, 9, 0, 9],
        [4, 2, 3, 5, 5],
        [2, 3, 7, 0, 2],
        [4, 2, 6, 4, 6],
        [2, 1, 5, 2, 3]],

       ...,

       [[4, 1, 9, 1, 5],
        [6, 4, 2, 7, 4],
        [2, 2, 0, 3, 8],
        [0, 7, 4, 6, 8],
        [1, 1, 1, 0, 4]],

       [[8, 8, 9, 0, 9],
        [9, 6, 9, 6, 1],
        [1, 7, 6, 7, 1],
        [6, 8, 0, 0, 3],
        [6, 2, 7, 5, 5]],

       [[7, 4, 5, 2, 3],
        [8, 5, 6, 1, 0],
        [7, 3, 7, 7, 4],
        [3, 1, 2, 7, 3],
        [4, 2, 7, 9, 0]]])

In [209]:
get_message_from_queue(queue_url)

KeyError: 'Messages'

In [204]:
# # sqs = boto3.resource('sqs')
# queue = sqs.(QueueName='airscholar-queue')
# for message in sqs.receive_message(
#             MaxNumberOfMessages=10):
#         # process message body
#         body = json.loads(message.body)
#         print(body)


In [143]:
# message = response['Messages']
# receipt_handle = message['ReceiptHandle']

# # Delete received message from queue
# sqs.delete_message(
#     QueueUrl=queue_url,
#     ReceiptHandle=receipt_handle
# )


In [196]:
# print(json.dumps(message, indent=4))

In [45]:
# # ec2_inst_id
# bucketName = 'airscholar-mlbd-bucket'

In [146]:
# s3 = boto3.client('s3')

In [ ]:
# response = s3.get_object(Bucket=bucketName,
#                          Key='data.json')
# print("Done, response body:")
# print(response['Body'].read())

In [ ]:
#create instance
#configure instance
#create matrix
#split matrix
#send matrix to the queue
#read matrix from the queue on the instance created
#compute matrix
#send result to base
